https://www.amazon.cn/gp/bestsellers/books

In [1]:
from IPython.display import HTML
import requests
r = requests.get('https://www.amazon.cn/gp/bestsellers/books')
open('amazonbook.html', 'w').write(r.text)
HTML('''
<iframe src="amazonbook.html" width=100% height=500px>
''')

This is a static page (frontpage). However, it uses "URI fragment" for pagination. i.e. the content after `#`. This piece of string is not sent to server or parsed by the server. It is analysed by browser so some javascript will trigger loading of more data. This is a typical case where you need to handle dynamic crawling.

Page 4 URL looks like this: https://www.amazon.cn/gp/bestsellers/books#4

In [2]:
r = requests.get('https://www.amazon.cn/gp/bestsellers/books#4')
open('amazonbook.html', 'w').write(r.text)
HTML('''
<iframe src="amazonbook.html" width=100% height=500px>
''')

You always see the first page because `requests.get` just ignore the parameter after `#`

In [3]:
import splinter

In [4]:
from splinter import Browser
import time

browser = Browser()
# Visit URL
url = "https://www.amazon.cn/gp/bestsellers/books#4"
browser.visit(url)
# You need to wait for some time for the page to be fully loaded
time.sleep(2)

In [5]:
#browser.html

In [6]:
#open('amazonbook.html', 'w').write(browser.html)
#HTML('''
#<iframe src="amazonbook.html" width=100% height=500px>
#''')

In [7]:
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
mypage = BeautifulSoup(browser.html)

/Users/hupili/Desktop/project/python-for-data-and-media-communication/venv/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [9]:
items = []
for row in mypage.find_all('div', attrs={'class': 'zg_itemRow'}):
    item = {}
    item['No.'] = row.find('span', attrs={'class': 'zg_rankNumber'}).text.strip()
    item['title'] = row.find('div', attrs={'class': 'p13n-sc-truncated'}).text.strip()
    item['author'] = row.find('div', attrs={'class': 'a-row'}).text.strip()
    items.append(item)
df = pd.DataFrame(items)
df

,No.,author,title
0,61.,简·尼尔森 (Jane Nelsen),正面管教(修订版)
1,62.,尤瓦尔·赫拉利,人类简史+未来简史(新版)(套装共2册)(随机附送思维导图和音频)
2,63.,埃米尼亚·伊贝拉 (Herminia Ibarra),逆向管理:先行动后思考
3,64.,尤瓦尔·赫拉利,人类简史:从动物到上帝(简体中文版)
4,65.,美国迪士尼公司,迪士尼我的第一本大百科全书
5,66.,英国DK公司,DK奇趣百科全书:是真是假?
6,67.,布兰达·S.迈尔斯 (Brenda S.Miles),儿童情绪管理与性格培养绘本·不怕犯错再试试:在错误中学会成长
7,68.,东野圭吾,白夜行(2017版)
8,69.,芭芭拉凯因(Barbara Cain),儿童情绪与人格培养绘本:我要更勇敢(中英对照)
9,70.,斯蒂文·谢尔弗,"美国儿科学会育儿百科(第6版,2018全新增订)"


In [10]:
# Call the following line to quit browser.
# If you use the "with" statement, the browser will be automatically closed when leaving that scope
browser.quit()

In [11]:
def scrape_one_page(url):
    items = []
    with Browser() as browser:
        # Visit URL
        #url = "https://www.amazon.cn/gp/bestsellers/books#4"
        browser.visit(url)
        # You need to wait for some time for the page to be fully loaded
        time.sleep(2)
        
        for row in mypage.find_all('div', attrs={'class': 'zg_itemRow'}):
            item = {}
            item['No.'] = row.find('span', attrs={'class': 'zg_rankNumber'}).text.strip()
            item['title'] = row.find('div', attrs={'class': 'p13n-sc-truncated'}).text.strip()
            item['author'] = row.find('div', attrs={'class': 'a-row'}).text.strip()
            items.append(item)
    return items


In [12]:
all_page_items = []
for i in range(5):
    new_items = scrape_one_page('https://www.amazon.cn/gp/bestsellers/books#%s' % i)
    all_page_items.extend(new_items)

When you execute the above code, you will see your browser pop up 5 times!

In [13]:
df = pd.DataFrame(all_page_items)
df

,No.,author,title
0,61.,简·尼尔森 (Jane Nelsen),正面管教(修订版)
1,62.,尤瓦尔·赫拉利,人类简史+未来简史(新版)(套装共2册)(随机附送思维导图和音频)
2,63.,埃米尼亚·伊贝拉 (Herminia Ibarra),逆向管理:先行动后思考
3,64.,尤瓦尔·赫拉利,人类简史:从动物到上帝(简体中文版)
4,65.,美国迪士尼公司,迪士尼我的第一本大百科全书
5,66.,英国DK公司,DK奇趣百科全书:是真是假?
6,67.,布兰达·S.迈尔斯 (Brenda S.Miles),儿童情绪管理与性格培养绘本·不怕犯错再试试:在错误中学会成长
7,68.,东野圭吾,白夜行(2017版)
8,69.,芭芭拉凯因(Barbara Cain),儿童情绪与人格培养绘本:我要更勇敢(中英对照)
9,70.,斯蒂文·谢尔弗,"美国儿科学会育儿百科(第6版,2018全新增订)"


In [14]:
len(df)

100